# 3.1 安装selenium并学习

In [ ]:
import selenium as sm
help(sm)

In [ ]:
# 模拟百度搜索

import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

wd = webdriver.Chrome()
wd.get("https://www.baidu.com")    # 打开百度浏览器
wd.find_element_by_id("kw").send_keys("selenium")    # 定位输入框并输入关键字
wd.find_element_by_id("su").click()    # 点击[百度一下]搜索  
time.sleep(3)    # 等待3秒
wd.quit()    # 关闭浏览器

In [ ]:
# 使用selenium模拟登陆163邮箱

import time
from selenium import webdriver


browser = webdriver.Chrome()
url = 'http://mail.163.com/'
browser.get(url)
time.sleep(3)

browser.maximize_window()    # 打开网页窗口

time.sleep(5)

browser.switch_to.frame(0)    # 找到邮箱账号登录框对应的iframe,由于网页中iframe的id是动态的，所以不能用id寻找

email = browser.find_element_by_name('email')    # 找到邮箱账号输入框

email.send_keys('******@163.com')    # 将自己的邮箱地址输入到邮箱账号框中
    
password = browser.find_element_by_name('password')    # 找到密码输入框
    
password.send_keys('******')    # 输入自己的邮箱密码
    
login_em = browser.find_element_by_id('dologin')    # 找到登陆按钮
    
login_em.click()    # 点击登陆按钮
      
time.sleep(10)

# 3.2 学习IP相关知识

### 3.2.1 获取代理IP地址

从该网站获取： https://www.xicidaili.com/ inspect -> 鼠标定位： 要获取的代理IP地址，属于class = "odd"标签的内容： 通过bs4的find_all(‘tr’)来获取所有IP

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

def open_proxy_url(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'
    headers = {'User-Agent': user_agent}
    try:
        r = requests.get(url, headers = headers, timeout = 20)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        print('无法访问网页' + url)


def get_proxy_ip(response):
    proxy_ip_list = []
    soup = BeautifulSoup(response, 'html.parser')
    proxy_ips = soup.find(id = 'ip_list').find_all('tr')
    for proxy_ip in proxy_ips:
        if len(proxy_ip.select('td')) >=8:
            ip = proxy_ip.select('td')[1].text
            port = proxy_ip.select('td')[2].text
            protocol = proxy_ip.select('td')[5].text
            if protocol in ('HTTP','HTTPS','http','https'):
                proxy_ip_list.append(f'{protocol}://{ip}:{port}')
    return proxy_ip_list

if __name__ == '__main__':
    proxy_url = 'https://www.xicidaili.com/'
    text = open_proxy_url(proxy_url)
    proxy_ip_filename = 'proxy_ip.txt'
    with open(proxy_ip_filename, 'w') as f:
        f.write(text)
    text = open(proxy_ip_filename, 'r').read()
    proxy_ip_list = get_proxy_ip(text)
    for i in proxy_ip_list:
        print(i)
    print(len(proxy_ip_list))

### 3.2.2 使用代理


proxies的格式是一个字典： {‘http’: ‘http://IP:port‘,‘https’:'https://IP:port‘} 把它直接传入requests的get方法中即可 web_data = requests.get(url, headers=headers, proxies=proxies)


### 3.2.3 确认代理IP地址有效性的三种方法


无论是免费还是收费的代理网站，提供的代理IP都未必有效，我们应该验证一下，有效后，再放入我们的代理IP池中，以下通过几种方式：

    验证返回码。访问网站，得到的返回码是200，则可用

    真正的访问某些网站，获取title等，验证title与预计的相同

    访问某些可以提供被访问IP的网站，类似于“查询我的IP”的网站，查看返回的IP地址是什么

In [ ]:
# 返回验证码

def check_proxy_avaliability_return(proxy):
    url = 'http://www.baidu.com'
    result = open_url_using_proxy(url, proxy)
    VALID_PROXY = False
    if result:
        text, status_code = result
        if status_code == 200:
            print('有效代理IP: ' + proxy)
        else:
            print('无效代理IP: ' + proxy)

In [ ]:
# 确认title

def check_proxy_avaliability_title(proxy):
    url = 'http://www.baidu.com'
    text, status_code = open_url_using_proxy(url, proxy)
    VALID = False
    if status_code == 200:
        if r_title:
            if r_title[0] == '<title>百度一下，你就知道</title>':
                VALID = True
    if VALID:
        print('有效代理IP: ' + proxy)
    else:
        print('无效代理IP: ' + proxy)